In [30]:
# https://colab.research.google.com/github/crux82/ganbert-pytorch/blob/main/GANBERT_pytorch.ipynb

# GAN-BERT (in Pytorch and compatible with HuggingFace)

This is a Pytorch (+ **Huggingface** transformers) implementation of the GAN-BERT model from https://github.com/crux82/ganbert. While the original GAN-BERT was an extension of BERT, this implementation can be adapted to several architectures, ranging from Roberta to Albert!

**NOTE**: given that this implementation is different from the original one in Tensorflow, some results can be slighty different.


Let's GO!

Required Imports.

In [536]:
!pip install -r /data/fire.txt
# !pip install tensorflow
# !conda install -c conda-forge -y pandas jupyter
# !pip install sacremoses
# !pip install -U numpy

# !conda install numpy
# !conda install -c intel mkl

# !export XRT_TPU_CONFIG="localservice;0;localhost:51011"
!echo $XRT_TPU_CONFIG

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

  Using cached https://storage.googleapis.com/cloud-tpu-tpuvm-artifacts/wheels/libtpu-nightly/libtpu_nightly-0.1.dev20211013-py3-none-any.whl (138.9 MB)
  Using cached lm_eval-0.2.0-py3-none-any.whl
  Using cached https://github.com/google-research/bleurt/archive/b610120347ef22b494b6d69b4316e303f5932516.zip
"localservice;0;localhost:51011"


In [537]:
!export XLA_USE_XRT=1 XRT_DEVICE_MAP="CPU:0;/job:localservice/replica:0/task:0/device:XLA_CPU:0"

!export TPU_NAME=life
!export TF_CPP_MIN_LOG_LEVEL=0
!export XRT_TPU_CONFIG="localservice;0;localhost:51011"
!export TF_XLA_FLAGS=--tf_xla_enable_xla_devices

# export XRT_DEVICE_MAP="CPU:0;/job:localservice/replica:0/task:0/device:XLA_CPU:0"
# export XRT_WORKERS="localservice:0;grpc://localhost:40934"

In [538]:
# !pip install sentencepiece
# !pip install transformers==4.3.2

import torch
import io
import torch.nn.functional as F
import random
import numpy as np
import time
import math
import datetime
import torch.nn as nn
from transformers import *
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
#!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html


##Set random values
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(seed_val)

RuntimeError: Failed to import transformers.convert_graph_to_onnx because of the following error (look up to see its traceback):
/root/anaconda3/envs/pytorch/lib/python3.8/site-packages/_XLAC.cpython-38-x86_64-linux-gnu.so: undefined symbol: _ZNK5torch4lazy4Node6shapesEv

In [539]:
# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


### Input Parameters


In [540]:
#--------------------------------
#  Transformer parameters
#--------------------------------
max_seq_length = 64
batch_size = 64

#--------------------------------
#  GAN-BERT specific parameters
#--------------------------------
# number of hidden layers in the generator, 
# each of the size of the output space
num_hidden_layers_g = 1; 
# number of hidden layers in the discriminator, 
# each of the size of the input space
num_hidden_layers_d = 1; 
# size of the generator's input noisy vectors
noise_size = 100
# dropout to be applied to discriminator's input vectors
out_dropout_rate = 0.2

# Replicate labeled data to balance poorly represented datasets, 
# e.g., less than 1% of labeled material
apply_balance = True

#--------------------------------
#  Optimization parameters
#--------------------------------
learning_rate_discriminator = 5e-5
learning_rate_generator = 5e-5
epsilon = 1e-8
num_train_epochs = 10
multi_gpu = True
# Scheduler
apply_scheduler = False
warmup_proportion = 0.1
# Print
print_each_n_step = 10

#--------------------------------
#  Adopted Tranformer model
#--------------------------------
# Since this version is compatible with Huggingface transformers, you can uncomment
# (or add) transformer models compatible with GAN

model_name = "bert-base-cased"
#model_name = "bert-base-uncased"
#model_name = "roberta-base"
#model_name = "albert-base-v2"
#model_name = "xlm-roberta-base"
#model_name = "amazon/bort"

#--------------------------------
#  Retrieve the TREC QC Dataset
#--------------------------------
# ! git clone https://github.com/crux82/ganbert

#  NOTE: in this setting 50 classes are involved
labeled_file = "./ganbert/data/labeled.tsv" #files for reviews are with _ta1
unlabeled_file = "./ganbert/data/unlabeled.tsv"
test_filename = "./ganbert/data/test.tsv"

#label_list = ["UNK", "Low", "High"]

#["UNK_UNK","ABBR_abb", "ABBR_exp", "DESC_def", "DESC_desc", 
              #"DESC_manner", "DESC_reason", "ENTY_animal", "ENTY_body", 
              #"ENTY_color", "ENTY_cremat", "ENTY_currency", "ENTY_dismed", 
              #"ENTY_event", "ENTY_food", "ENTY_instru", "ENTY_lang", 
              #"ENTY_letter", "ENTY_other", "ENTY_plant", "ENTY_product", 
              #"ENTY_religion", "ENTY_sport", "ENTY_substance", "ENTY_symbol", 
              #"ENTY_techmeth", "ENTY_termeq", "ENTY_veh", "ENTY_word", "HUM_desc", 
              #"HUM_gr", "HUM_ind", "HUM_title", "LOC_city", "LOC_country", 
              #"LOC_mount", "LOC_other", "LOC_state", "NUM_code", "NUM_count", 
              #"NUM_date", "NUM_dist", "NUM_money", "NUM_ord", "NUM_other", 
              #"NUM_perc", "NUM_period", "NUM_speed", "NUM_temp", "NUM_volsize", 
              #"NUM_weight"] 

In [549]:
import pandas as pd
df_lab = pd.read_csv("./ganbert/data/labeled_3lab.csv")
df_unlab = pd.read_csv("./ganbert/data/unlabeled_ta4607.csv")#ta4607
df_test = pd.read_csv("./ganbert/data/test_3lab.csv")

#only if sentiment
#indexNames = df_lab[~(df_lab['pos_neg'] == ':POS')].index 
#df_lab.drop(indexNames , inplace=True)
#indexNames = df_test[~(df_test['pos_neg'] == ':POS')].index 
#df_test.drop(indexNames , inplace=True)

#df_lab['label'] = df_lab['Label']
#df_unlab['label'] = df_unlab['Label']
#df_test['label'] = df_test['Label']

#df_lab['review_text'] = df_lab['Review Text']
#df_unlab['review_text'] = df_unlab['Review Text']
#df_test['review_text'] = df_test['Review Text']

df_lab['label'] = df_lab['quality']
df_unlab['label'] = df_unlab['quality']
df_test['label'] = df_test['quality']

#df_lab = df_lab.sample(n=100)
#df_unlab = df_unlab.sample(n=4000)
#df_test = df_test.sample(n=500)

df_lab

,url,hotel_name,review_title,review_text,helpful_count,publish_date,date_today,days_published,word_count,quality,label
0,https://www.tripadvisor.ca/ShowUserReviews-g22...,"Holiday Inn Express Amsterdam - Schiphol, an I...",A lovely place at a great price,"This hotel has great rooms, lovely breakfast, ...",0,2015-08-05,2022-05-19,2479.0,573,Low,Low
1,https://www.tripadvisor.ca/ShowUserReviews-g22...,Hyatt Place Amsterdam Airport,"It would have 5 star, if it was closer to the ...",Airportの名前がついているものの、空港からは車で10分ほどかかる（ホテルの空港送迎もあ...,0,2017-05-02,2022-05-19,1843.0,360,Low,Low
2,https://www.tripadvisor.ca/ShowUserReviews-g22...,Van der Valk Hotel Schiphol,Great place to stay near the airport.,"My son, daughter and I stayed in a triple room...",4,2015-09-01,2022-05-19,2452.0,725,Medium,Medium
3,https://www.tripadvisor.ca/ShowUserReviews-g18...,Steigenberger Airport Hotel Amsterdam,Perhaps best value for money in that area,"Good restaurants and bars, running road throug...",1,2016-09-23,2022-05-19,2064.0,275,Medium,Medium
4,https://www.tripadvisor.ca/ShowUserReviews-g18...,Hotel Mitland,"Nice hotel, good service",I have stayed in the mitland hotel several tim...,7,2004-09-30,2022-05-19,6440.0,533,High,High
...,...,...,...,...,...,...,...,...,...,...,...
205,https://www.tripadvisor.ca/ShowUserReviews-g22...,Van der Valk Hotel Schiphol,Good hotel,Everything is just fine! Breakfast is very var...,0,2015-03-10,2022-05-19,2627.0,266,Low,Low
206,https://www.tripadvisor.ca/ShowUserReviews-g18...,Element Amsterdam,Worst hotel ever stayed,I have newer stayed such a worst hotel. The st...,0,2019-11-29,2022-05-19,902.0,226,Low,Low
207,https://www.tripadvisor.ca/ShowUserReviews-g18...,City Hotel Groningen,Disappointing based on reviews here,I booked this hotel for an overnight stay in G...,5,2011-02-20,2022-05-19,4106.0,1375,High,High
208,https://www.tripadvisor.ca/ShowUserReviews-g18...,Babylon Hotel Den Haag,Conveniently located good hotel!,Stayed here on business for roughly a week. Th...,1,2014-02-03,2022-05-19,3027.0,568,Medium,Medium


In [550]:
#remove olk
df_lab = df_lab[df_lab['days_published'] > 730] #unkown labelled are recently published because not a lot of time to acquire upvotes
df_test = df_test[df_test['days_published'] > 730]

df_lab['days_published'].describe()

count     198.000000
mean     2908.929293
std      1600.336613
min       839.000000
25%      1771.250000
50%      2548.000000
75%      3554.250000
max      6809.000000
Name: days_published, dtype: float64

In [551]:
#check for label list
print(df_lab.label.unique())
print(df_unlab.label.unique())
print(df_test.label.unique())

['Low' 'Medium' 'High']
['UNK']
['Medium' 'Low' 'High']


In [552]:
df_test['label'].value_counts()

High      332
Medium    319
Low       319
Name: label, dtype: int64

In [553]:
#add labels
label_list = ['UNK','High', 'Low', 'Medium']

In [554]:
df_lab['label'].value_counts()

High      69
Medium    66
Low       63
Name: label, dtype: int64

In [555]:
df_test['label'].value_counts()

High      332
Medium    319
Low       319
Name: label, dtype: int64

In [556]:
print(df_lab.shape)
print(df_unlab.shape)
print(df_test.shape)

(198, 11)
(4607, 3)
(970, 12)


In [557]:
print(df_lab.index)

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            198, 200, 202, 203, 204, 205, 206, 207, 208, 209],
           dtype='int64', length=198)


In [567]:
# Defining function to format the dataset to be used in the dataloader
def get_examples(df):

    examples = []

    # iterate through each row
    for index, row in df.iterrows():
        examples.append((row['review_text'], row['label']))

    return examples

In [568]:
labeled_examples = get_examples(df_lab)
unlabeled_examples = get_examples(df_unlab)
test_examples = get_examples(df_test)

In [569]:
labeled_examples

[("This hotel has great rooms, lovely breakfast, superb location (easy access to train to Amsterdam and airport shuttle), and wonderful service. ,The room was very modern in style. The air conditioning worked well. The hotel was well-maintained. ,The continental breakfast had a selection of breads and meats/cheeses, etc. There's also a bar, ice machine, and other convenience goods sold. ,The best part was how conveniently located this place is. An airport shuttle every 40 minutes and a 10-minute walk to the train station. ,The staff was helpful. I would stay here again.",
  'Low'),
 ('Airportの名前がついているものの、空港からは車で10分ほどかかる（ホテルの空港送迎もあり。でもすぐに定員がいっぱいになる。次まで15分ほど待たなければならない）。なので中心地に行くにも、空港に行くにもなんだかんだと時間がかかり、短い滞在には動き方を考えないと時間のロスになります。空港周辺にはUberがウロウロしているので,それで当家は対応。,ただその点を除けば、宿泊料金はvalue、シンプルで非常に綺麗で広々したお部屋、窓からの眺めも良い（周りに何もないから、、、笑）、静か（これも周りに何もないから、、、）ネットは安定している、朝食もついている、スタッフの応対も良いなど特に不満を感じるところはありませんでした。お水やお酒は少し割高ですが、１階のバーで深夜まで買えます。子連れでも安心して過ごせるホテルです。',
  'Low'),
 ('My son, daughter and I stayed in

Load the Tranformer Model

In [570]:
!echo $XRT_TPU_CONFIG

"localservice;0;localhost:51011"


In [571]:
from transformers import AutoConfig, AutoModel, AutoModelForCausalLM, AutoTokenizer

transformer = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Function required to load the dataset

In [572]:
#not needed to run
#def get_qc_examples(input_file):
#  """Creates examples for the training and dev sets."""
#  examples = []
#
#  with open(input_file, 'r') as f:
#      contents = f.read()
#      file_as_list = contents.splitlines()
#      for line in file_as_list[1:]:
#          split = line.split(" ")
#          question = ' '.join(split[1:])
#
#          text_a = question
#          inn_split = split[0].split(":")
#          label = inn_split[0] + "_" + inn_split[1]
#          examples.append((text_a, label))
#      f.close()
#
#  return examples

**Load** the input QC dataset (fine-grained)

In [573]:
#not needed to run
#Load the examples
#labeled_examples = get_qc_examples(labeled_file)
#unlabeled_examples = get_qc_examples(unlabeled_file)
#test_examples = get_qc_examples(test_filename)

Functions required to convert examples into Dataloader

In [574]:
def generate_data_loader(input_examples, label_masks, label_map, do_shuffle = False, balance_label_examples = False):
  '''
  Generate a Dataloader given the input examples, eventually masked if they are 
  to be considered NOT labeled.
  '''
  examples = []

  # Count the percentage of labeled examples  
  num_labeled_examples = 0
  for label_mask in label_masks:
    if label_mask: 
      num_labeled_examples += 1
  label_mask_rate = num_labeled_examples/len(input_examples)

  # if required it applies the balance
  for index, ex in enumerate(input_examples): 
    if label_mask_rate == 1 or not balance_label_examples:
      examples.append((ex, label_masks[index]))
    else:
      # IT SIMULATE A LABELED EXAMPLE
      if label_masks[index]:
        balance = int(1/label_mask_rate)
        balance = int(math.log(balance,2))
        if balance < 1:
          balance = 1
        for b in range(0, int(balance)):
          examples.append((ex, label_masks[index]))
      else:
        examples.append((ex, label_masks[index]))
  
  #-----------------------------------------------
  # Generate input examples to the Transformer
  #-----------------------------------------------
  input_ids = []
  input_mask_array = []
  label_mask_array = []
  label_id_array = []

  # Tokenization 
  for (text, label_mask) in examples:
    encoded_sent = tokenizer.encode(text[0], add_special_tokens=True, max_length=max_seq_length, padding="max_length", truncation=True)
    input_ids.append(encoded_sent)
    label_id_array.append(label_map[text[1]])
    label_mask_array.append(label_mask)
  
  # Attention to token (to ignore padded input wordpieces)
  for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]                          
    input_mask_array.append(att_mask)
  # Convertion to Tensor
  input_ids = torch.tensor(input_ids) 
  input_mask_array = torch.tensor(input_mask_array)
  label_id_array = torch.tensor(label_id_array, dtype=torch.long)
  label_mask_array = torch.tensor(label_mask_array)

  # Building the TensorDataset
  dataset = TensorDataset(input_ids, input_mask_array, label_id_array, label_mask_array)

  if do_shuffle:
    sampler = RandomSampler
  else:
    sampler = SequentialSampler

  # Building the DataLoader
  return DataLoader(
              dataset,  # The training samples.
              sampler = sampler(dataset), 
              batch_size = batch_size) # Trains with this batch size.

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

Convert the input examples into DataLoader

In [575]:
# !pip install tensorflow_datasets
# import tensorflow as tf
# import tensorflow_datasets as tfds

# import torch
# import tensorflow
# import TensorData from 'tensorfow'

import torch
from torch.utils.data import TensorDataset
from torch.utils.data import RandomSampler
from torch.utils.data import DataLoader
from torch.utils.data import SequentialSampler

label_map = {}
for (i, label) in enumerate(label_list):
  label_map[label] = i
#------------------------------
#   Load the train dataset
#------------------------------
train_examples = labeled_examples
#The labeled (train) dataset is assigned with a mask set to True
train_label_masks = np.ones(len(labeled_examples), dtype=bool)
#If unlabel examples are available
if unlabeled_examples:
  train_examples = train_examples + unlabeled_examples
  #The unlabeled (train) dataset is assigned with a mask set to False
  tmp_masks = np.zeros(len(unlabeled_examples), dtype=bool)
  train_label_masks = np.concatenate([train_label_masks,tmp_masks])

train_dataloader = generate_data_loader(train_examples, train_label_masks, label_map, do_shuffle = True, balance_label_examples = apply_balance)

#------------------------------
#   Load the test dataset
#------------------------------
#The labeled (test) dataset is assigned with a mask set to True
test_label_masks = np.ones(len(test_examples), dtype=bool)

test_dataloader = generate_data_loader(test_examples, test_label_masks, label_map, do_shuffle = False, balance_label_examples = False)

<ipython-input-574-b5617700c116>:54: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  label_mask_array = torch.tensor(label_mask_array)


We define the Generator and Discriminator as discussed in https://www.aclweb.org/anthology/2020.acl-main.191/

In [576]:
#------------------------------
#   The Generator as in 
#   https://www.aclweb.org/anthology/2020.acl-main.191/
#   https://github.com/crux82/ganbert
#------------------------------
class Generator(nn.Module):
    def __init__(self, noise_size=100, output_size=512, hidden_sizes=[512], dropout_rate=0.1):
        super(Generator, self).__init__()
        layers = []
        hidden_sizes = [noise_size] + hidden_sizes
        for i in range(len(hidden_sizes)-1):
            layers.extend([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]), nn.LeakyReLU(0.2, inplace=True), nn.Dropout(dropout_rate)])

        layers.append(nn.Linear(hidden_sizes[-1],output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, noise):
        output_rep = self.layers(noise)
        return output_rep

#------------------------------
#   The Discriminator
#   https://www.aclweb.org/anthology/2020.acl-main.191/
#   https://github.com/crux82/ganbert
#------------------------------
class Discriminator(nn.Module):
    def __init__(self, input_size=512, hidden_sizes=[512], num_labels=2, dropout_rate=0.1):
        super(Discriminator, self).__init__()
        self.input_dropout = nn.Dropout(p=dropout_rate)
        layers = []
        hidden_sizes = [input_size] + hidden_sizes
        for i in range(len(hidden_sizes)-1):
            layers.extend([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]), nn.LeakyReLU(0.2, inplace=True), nn.Dropout(dropout_rate)])

        self.layers = nn.Sequential(*layers) #per il flatten
        self.logit = nn.Linear(hidden_sizes[-1],num_labels+1) # +1 for the probability of this sample being fake/real.
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, input_rep):
        input_rep = self.input_dropout(input_rep)
        last_rep = self.layers(input_rep)
        logits = self.logit(last_rep)
        probs = self.softmax(logits)
        return last_rep, logits, probs

We instantiate the Discriminator and Generator

In [577]:
# The config file is required to get the dimension of the vector produced by 
# the underlying transformer
config = AutoConfig.from_pretrained(model_name)
hidden_size = int(config.hidden_size)
# Define the number and width of hidden layers
hidden_levels_g = [hidden_size for i in range(0, num_hidden_layers_g)]
hidden_levels_d = [hidden_size for i in range(0, num_hidden_layers_d)]

#-------------------------------------------------
#   Instantiate the Generator and Discriminator
#-------------------------------------------------
generator = Generator(noise_size=noise_size, output_size=hidden_size, hidden_sizes=hidden_levels_g, dropout_rate=out_dropout_rate)
discriminator = Discriminator(input_size=hidden_size, hidden_sizes=hidden_levels_d, num_labels=len(label_list), dropout_rate=out_dropout_rate)

# Put everything in the GPU if available
if torch.cuda.is_available():    
  generator.cuda()
  discriminator.cuda()
  transformer.cuda()
  if multi_gpu:
    transformer = torch.nn.DataParallel(transformer)

# print(config)

Let's go with the training procedure

In [578]:
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

#models parameters
transformer_vars = [i for i in transformer.parameters()]
d_vars = transformer_vars + [v for v in discriminator.parameters()]
g_vars = [v for v in generator.parameters()]

#optimizer
dis_optimizer = torch.optim.AdamW(d_vars, lr=learning_rate_discriminator)
gen_optimizer = torch.optim.AdamW(g_vars, lr=learning_rate_generator) 

#scheduler
if apply_scheduler:
  num_train_examples = len(train_examples)
  num_train_steps = int(num_train_examples / batch_size * num_train_epochs)
  num_warmup_steps = int(num_train_steps * warmup_proportion)

  scheduler_d = get_constant_schedule_with_warmup(dis_optimizer, 
                                           num_warmup_steps = num_warmup_steps)
  scheduler_g = get_constant_schedule_with_warmup(gen_optimizer, 
                                           num_warmup_steps = num_warmup_steps)

# For each epoch...
for epoch_i in range(0, num_train_epochs):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, num_train_epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    tr_g_loss = 0
    tr_d_loss = 0

    # Put the model into training mode.
    transformer.train() 
    generator.train()
    discriminator.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every print_each_n_step batches.
        if step % print_each_n_step == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        b_label_mask = batch[3].to(device)

        real_batch_size = b_input_ids.shape[0]
     
        # Encode real data in the Transformer
        model_outputs = transformer(b_input_ids, attention_mask=b_input_mask)
        hidden_states = model_outputs[-1]
        
        # Generate fake data that should have the same distribution of the ones
        # encoded by the transformer. 
        # First noisy input are used in input to the Generator
        noise = torch.zeros(real_batch_size, noise_size, device=device).uniform_(0, 1)
        # Gnerate Fake data
        gen_rep = generator(noise)

        # Generate the output of the Discriminator for real and fake data.
        # First, we put together the output of the tranformer and the generator
        disciminator_input = torch.cat([hidden_states, gen_rep], dim=0)
        # Then, we select the output of the disciminator
        features, logits, probs = discriminator(disciminator_input)

        # Finally, we separate the discriminator's output for the real and fake
        # data
        features_list = torch.split(features, real_batch_size)
        D_real_features = features_list[0]
        D_fake_features = features_list[1]
      
        logits_list = torch.split(logits, real_batch_size)
        D_real_logits = logits_list[0]
        D_fake_logits = logits_list[1]
        
        probs_list = torch.split(probs, real_batch_size)
        D_real_probs = probs_list[0]
        D_fake_probs = probs_list[1]

        #---------------------------------
        #  LOSS evaluation
        #---------------------------------
        # Generator's LOSS estimation
        g_loss_d = -1 * torch.mean(torch.log(1 - D_fake_probs[:,-1] + epsilon))
        g_feat_reg = torch.mean(torch.pow(torch.mean(D_real_features, dim=0) - torch.mean(D_fake_features, dim=0), 2))
        g_loss = g_loss_d + g_feat_reg
  
        # Disciminator's LOSS estimation
        logits = D_real_logits[:,0:-1]
        log_probs = F.log_softmax(logits, dim=-1)
        # The discriminator provides an output for labeled and unlabeled real data
        # so the loss evaluated for unlabeled data is ignored (masked)
        label2one_hot = torch.nn.functional.one_hot(b_labels, len(label_list))
        per_example_loss = -torch.sum(label2one_hot * log_probs, dim=-1)
        per_example_loss = torch.masked_select(per_example_loss, b_label_mask.to(device))
        labeled_example_count = per_example_loss.type(torch.float32).numel()

        # It may be the case that a batch does not contain labeled examples, 
        # so the "supervised loss" in this case is not evaluated
        if labeled_example_count == 0:
          D_L_Supervised = 0
        else:
          D_L_Supervised = torch.div(torch.sum(per_example_loss.to(device)), labeled_example_count)
                 
        D_L_unsupervised1U = -1 * torch.mean(torch.log(1 - D_real_probs[:, -1] + epsilon))
        D_L_unsupervised2U = -1 * torch.mean(torch.log(D_fake_probs[:, -1] + epsilon))
        d_loss = D_L_Supervised + D_L_unsupervised1U + D_L_unsupervised2U

        #---------------------------------
        #  OPTIMIZATION
        #---------------------------------
        # Avoid gradient accumulation
        gen_optimizer.zero_grad()
        dis_optimizer.zero_grad()

        # Calculate weigth updates
        # retain_graph=True is required since the underlying graph will be deleted after backward
        g_loss.backward(retain_graph=True)
        d_loss.backward() 
        
        # Apply modifications
        gen_optimizer.step()
        dis_optimizer.step()

        # A detail log of the individual losses
        #print("{0:.4f}\t{1:.4f}\t{2:.4f}\t{3:.4f}\t{4:.4f}".
        #      format(D_L_Supervised, D_L_unsupervised1U, D_L_unsupervised2U,
        #             g_loss_d, g_feat_reg))

        # Save the losses to print them later
        tr_g_loss += g_loss.item()
        tr_d_loss += d_loss.item()

        # Update the learning rate with the scheduler
        if apply_scheduler:
          scheduler_d.step()
          scheduler_g.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss_g = tr_g_loss / len(train_dataloader)
    avg_train_loss_d = tr_d_loss / len(train_dataloader)             
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss generetor: {0:.3f}".format(avg_train_loss_g))
    print("  Average training loss discriminator: {0:.3f}".format(avg_train_loss_d))
    print("  Training epoch took: {:}".format(training_time))
        
    # ========================================
    #     TEST ON THE EVALUATION DATASET
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our test set.
    print("")
    print("Running Test...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    transformer.eval() #maybe redundant
    discriminator.eval()
    generator.eval()

    # Tracking variables 
    total_test_accuracy = 0
   
    total_test_loss = 0
    nb_test_steps = 0

    all_preds = []
    all_labels_ids = []

    #loss
    nll_loss = torch.nn.CrossEntropyLoss(ignore_index=-1)

    # Evaluate data for one epoch
    for batch in test_dataloader:
        
        # Unpack this training batch from our dataloader. 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        
            model_outputs = transformer(b_input_ids, attention_mask=b_input_mask)
            hidden_states = model_outputs[-1]
            _, logits, probs = discriminator(hidden_states)
            ###log_probs = F.log_softmax(probs[:,1:], dim=-1)
            filtered_logits = logits[:,0:-1]
            # Accumulate the test loss.
            total_test_loss += nll_loss(filtered_logits, b_labels)
            
        # Accumulate the predictions and the input labels
        _, preds = torch.max(filtered_logits, 1)
        all_preds += preds.detach().cpu()
        all_labels_ids += b_labels.detach().cpu()

    # Report the final accuracy for this validation run.
    all_preds = torch.stack(all_preds).numpy()
    all_labels_ids = torch.stack(all_labels_ids).numpy()
    test_accuracy = np.sum(all_preds == all_labels_ids) / len(all_preds)
    print("  Accuracy: {0:.3f}".format(test_accuracy))

    # Calculate the average loss over all of the batches.
    avg_test_loss = total_test_loss / len(test_dataloader)
    avg_test_loss = avg_test_loss.item()
    
    # Measure how long the validation run took.
    test_time = format_time(time.time() - t0)
    
    print("  Test Loss: {0:.3f}".format(avg_test_loss))
    print("  Test took: {:}".format(test_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss generator': avg_train_loss_g,
            'Training Loss discriminator': avg_train_loss_d,
            'Valid. Loss': avg_test_loss,
            'Valid. Accur.': test_accuracy,
            'Training Time': training_time,
            'Test Time': test_time
        }
    )


======== Epoch 1 / 10 ========
Training...
  Batch    10  of     85.    Elapsed: 0:01:04.
  Batch    20  of     85.    Elapsed: 0:01:44.
  Batch    30  of     85.    Elapsed: 0:02:29.
  Batch    40  of     85.    Elapsed: 0:03:12.
  Batch    50  of     85.    Elapsed: 0:03:53.
  Batch    60  of     85.    Elapsed: 0:04:37.
  Batch    70  of     85.    Elapsed: 0:05:17.
  Batch    80  of     85.    Elapsed: 0:06:02.

  Average training loss generetor: 0.696
  Average training loss discriminator: 1.927
  Training epoch took: 0:06:24

Running Test...
  Accuracy: 0.409
  Test Loss: 1.312
  Test took: 0:00:08

======== Epoch 2 / 10 ========
Training...
  Batch    10  of     85.    Elapsed: 0:00:43.
  Batch    20  of     85.    Elapsed: 0:01:26.
  Batch    30  of     85.    Elapsed: 0:02:09.
  Batch    40  of     85.    Elapsed: 0:02:52.
  Batch    50  of     85.    Elapsed: 0:03:34.
  Batch    60  of     85.    Elapsed: 0:04:17.
  Batch    70  of     85.    Elapsed: 0:05:00.
  Batch    80 

In [579]:
for stat in training_stats:
  print(stat)

print("\nTraining complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

{'epoch': 1, 'Training Loss generator': 0.6964774475378148, 'Training Loss discriminator': 1.9267785016228172, 'Valid. Loss': 1.3122949600219727, 'Valid. Accur.': 0.4092783505154639, 'Training Time': '0:06:24', 'Test Time': '0:00:08'}
{'epoch': 2, 'Training Loss generator': 0.7346591893364401, 'Training Loss discriminator': 0.9485033897792592, 'Valid. Loss': 2.0812172889709473, 'Valid. Accur.': 0.42577319587628865, 'Training Time': '0:06:00', 'Test Time': '0:00:08'}
{'epoch': 3, 'Training Loss generator': 0.7149927553008585, 'Training Loss discriminator': 0.7470719428623424, 'Valid. Loss': 2.5097291469573975, 'Valid. Accur.': 0.42061855670103093, 'Training Time': '0:05:54', 'Test Time': '0:00:08'}
{'epoch': 4, 'Training Loss generator': 0.7072820256738102, 'Training Loss discriminator': 0.7240233533522662, 'Valid. Loss': 3.152801513671875, 'Valid. Accur.': 0.42061855670103093, 'Training Time': '0:05:51', 'Test Time': '0:00:08'}
{'epoch': 5, 'Training Loss generator': 0.7030415261493009

In [580]:
pwd

'/data'

In [581]:
from sklearn.metrics import classification_report

print(classification_report(all_labels_ids, all_preds))

              precision    recall  f1-score   support

           1       0.51      0.24      0.33       332
           2       0.42      0.39      0.41       319
           3       0.34      0.55      0.42       319

    accuracy                           0.39       970
   macro avg       0.42      0.40      0.39       970
weighted avg       0.43      0.39      0.39       970



In [582]:
print(all_labels_ids) 

[3 2 3 2 1 1 3 1 2 1 3 2 2 1 2 2 1 2 3 1 1 3 3 3 2 2 2 3 1 1 1 1 1 1 3 2 1
 1 1 2 2 1 2 2 3 2 1 2 1 2 1 3 3 2 2 3 3 2 3 3 2 3 2 3 3 3 2 3 3 2 2 1 1 2
 3 1 2 2 3 3 3 2 1 3 1 3 1 2 2 1 1 1 3 2 3 3 2 3 1 3 1 2 1 2 2 1 3 1 3 1 1
 3 1 2 1 3 1 2 1 2 2 1 3 2 1 3 1 3 3 3 3 1 2 1 3 3 2 3 2 1 2 3 3 1 3 2 3 1
 3 2 1 1 2 2 3 2 1 1 3 1 1 3 3 1 3 1 1 1 1 1 3 3 3 2 3 1 1 2 3 3 3 3 2 2 2
 3 3 1 2 1 2 2 2 2 3 2 2 2 1 1 1 3 3 1 1 1 3 2 3 1 3 1 3 2 1 3 1 2 1 1 1 2
 2 3 3 2 2 1 2 2 1 2 3 1 2 2 2 2 3 3 2 3 1 2 3 1 1 1 2 1 3 2 2 2 3 1 2 3 2
 1 1 2 1 2 1 3 3 1 3 3 2 3 3 3 1 2 1 1 3 2 3 3 3 3 2 2 3 2 1 2 2 2 1 2 1 1
 3 2 2 2 3 1 3 1 3 3 3 1 2 3 1 3 2 2 1 2 2 1 2 1 1 1 1 1 2 3 1 2 3 3 2 1 2
 1 1 1 1 3 3 2 3 3 2 2 3 2 1 3 1 1 3 2 3 1 2 2 3 3 2 3 3 2 2 3 3 3 3 2 2 3
 1 3 2 2 1 1 1 1 1 2 1 1 1 1 2 3 2 1 3 3 3 3 2 3 1 3 1 1 2 2 2 3 3 2 2 1 1
 1 1 3 2 2 3 2 3 1 1 2 2 3 2 3 2 2 2 2 1 2 3 1 3 2 3 3 2 3 1 1 1 2 2 3 3 2
 3 3 3 3 2 3 3 1 2 2 1 1 3 3 1 2 3 3 3 2 1 2 1 1 1 3 1 1 1 3 2 2 1 3 2 1 1
 2 1 1 3 2 2 3 2 1 1 2 1 

In [583]:
print(all_preds)

[2 3 3 1 3 2 3 1 2 1 3 3 2 3 1 3 2 3 2 3 1 3 3 2 1 2 1 3 2 3 3 3 3 2 2 3 3
 3 2 2 3 1 2 2 3 3 3 3 3 3 3 1 2 3 3 3 3 3 3 3 2 3 1 3 2 3 2 1 3 2 2 2 2 2
 2 3 2 2 3 3 3 3 2 1 3 2 1 3 2 2 3 2 2 3 1 1 2 3 1 2 2 2 3 2 3 2 1 3 3 3 1
 3 1 2 3 2 3 2 1 3 2 2 2 2 2 3 1 3 3 3 3 3 3 3 2 2 2 2 2 1 3 3 2 1 2 2 1 3
 1 1 3 3 2 3 2 3 3 2 1 3 3 3 2 2 3 2 2 3 1 3 3 3 3 3 2 1 3 2 3 1 1 3 3 3 3
 3 3 3 2 3 3 1 2 2 3 3 3 1 3 1 3 2 3 2 1 3 2 2 3 2 2 1 3 3 2 1 2 2 1 3 3 3
 3 2 2 3 2 2 2 1 2 2 3 3 2 3 2 3 2 1 3 2 3 3 3 1 1 2 3 3 2 1 2 3 3 3 1 1 3
 3 3 2 1 3 2 1 3 1 1 2 2 3 3 3 1 3 3 2 2 2 3 2 3 2 2 3 1 2 2 3 3 2 3 2 3 2
 3 3 2 3 2 3 3 2 2 3 2 3 3 1 3 3 3 2 3 2 3 2 3 2 3 1 3 1 2 1 3 3 3 3 2 3 3
 3 3 1 3 3 1 1 3 3 2 3 3 3 1 3 3 1 3 2 2 1 3 3 3 3 3 1 3 3 3 3 1 1 3 3 2 2
 2 2 2 3 3 1 1 1 3 3 3 1 2 3 3 1 1 1 1 3 3 3 2 1 2 3 2 3 2 3 3 3 3 3 2 3 3
 2 2 2 3 3 2 3 3 3 1 2 3 1 3 2 2 3 2 3 3 1 1 3 2 2 2 3 2 3 3 3 3 2 2 1 1 3
 3 3 3 2 3 2 2 1 2 3 2 3 3 3 2 3 3 3 2 3 1 3 3 3 2 3 3 1 1 1 1 2 3 1 3 3 3
 2 2 3 3 2 3 3 3 3 2 3 2 